In [ ]:
first_time = True
full_data = True

copy_full_data = True
copy_sample_data = False

In [ ]:
if first_time:
  from google.colab import drive
  drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
if first_time:
  !pip install petastorm[torch]

     |████████████████████████████████| 286kB 7.8MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 20.7MB 1.5MB/s 
     |████████████████████████████████| 204.8MB 55kB/s 
     |████████████████████████████████| 204kB 36.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.2-py2.py3-none-any.whl size=205186687 sha256=854d37394a1c14019ea8dc9008016eb503accae15e1d45acd1387cc084b8d493
  Stored in directory: /root/.cache/pip/wheels/8b/09/da/c1f2859bcc86375dc972c5b6af4881b3603269bcc4c9be5d16
Successfully built pyspark
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [ ]:
if first_time:
  !apt-get update
  !apt-get install openjdk-8-jdk-headless -qq > /dev/null
  # !wget -q https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
  # !tar xf spark-2.4.7-bin-hadoop2.7.tgz
  !wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
  !tar xf spark-3.0.0-bin-hadoop2.7.tgz

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [1,929 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]

In [ ]:
# Set up Spark
if first_time:
  !pip install -q findspark
  !pip install py4j

!export JAVA_HOME=$(/usr/lib/jvm/java-8-openjdk-amd64 -v 1.8)
! echo $JAVA_HOME
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"

import findspark
# findspark.init("spark-2.4.7-bin-hadoop2.7")# SPARK_HOME
findspark.init("spark-3.0.0-bin-hadoop2.7")# SPARK_HOME
if first_time:
  !pip install --upgrade PyArrow
import os
os.environ["PYSPARK_PYTHON"] = "python3"
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"


from pyspark.sql import SparkSession

/bin/bash: /usr/lib/jvm/java-8-openjdk-amd64: Is a directory

Requirement already up-to-date: PyArrow in /usr/local/lib/python3.6/dist-packages (3.0.0)


In [ ]:
  spark = SparkSession \
      .builder \
      .appName("AvazuCTR") \
      .config("spark.some.config.option", "some-value") \
      .getOrCreate()

## Load processed data

In [ ]:
# copy parquet files from Google Drive
if copy_full_data:
  !cp -R /content/drive/MyDrive/LAI_OFFER/BIG_DATA/AvazuCTR/WDL_FTRL/data_wDash/train_df_parquet .
  !cp -R /content/drive/MyDrive/LAI_OFFER/BIG_DATA/AvazuCTR/WDL_FTRL/data_wDash/test_df_parquet .
if copy_sample_data:
  !cp -R /content/drive/MyDrive/LAI_OFFER/BIG_DATA/AvazuCTR/WDL_FTRL/data_wDash/train_df_parquet_filtered .
  !cp -R /content/drive/MyDrive/LAI_OFFER/BIG_DATA/AvazuCTR/WDL_FTRL/data_wDash/test_df_parquet_filtered .
  
# a larger number of partitions makes a partition smaller than the allocated excutor memory ensuring performance
# partition usually will be different fro where saved
# https://stackoverflow.com/questions/50825835/does-spark-maintain-parquet-partitioning-on-read
if full_data:
  train_df = spark.read.parquet("train_df_parquet")#.repartition(4) #repartition(64)
  test_df = spark.read.parquet("test_df_parquet")
else:
  train_df = spark.read.parquet("train_df_parquet_filtered")#.repartition(4) #repartition(64)
  test_df = spark.read.parquet("test_df_parquet_filtered")


In [ ]:
# repartition by 'date', lazy
from pyspark.sql.functions import col as column
train_df = train_df.repartition(9,column('date'))
test_df = test_df.repartition(9,column('date'))

In [ ]:
# data type conversion for 'id'
from pyspark.sql.types import FloatType,IntegerType,DoubleType
train_df = train_df.withColumn('id',  column('id').cast(DoubleType()))
test_df = test_df.withColumn('id',  column('id').cast(DoubleType()))
train_df.printSchema()

root
 |-- id: double (nullable = true)
 |-- hashed_wide: vector (nullable = true)
 |-- embedding_indexed: vector (nullable = true)
 |-- label: float (nullable = true)
 |-- date: integer (nullable = true)



In [ ]:
# sort by 'date', lazy, this ordering will be messed up by petastorm
# train_df = train_df.orderBy('date',ascending = True)
# test_df = test_df.orderBy('date',ascending = True)

In [ ]:
import pickle
path_processed_features = '/content/drive/MyDrive/LAI_OFFER/BIG_DATA/AvazuCTR/WDL_FTRL/data_wDash/1_SparkProcess_AvazuCTR.pickle'
with open(path_processed_features,'rb') as handle:  
  wide_cols = pickle.load(handle)
  interactive_features = pickle.load(handle)
  D = pickle.load(handle)
  embeddings_cols = pickle.load(handle)
  loader_cols = pickle.load(handle)
print('len wide_cols',len(wide_cols))
print('len interactive_features',len(interactive_features))
print('D',D)
print('len embeddings_cols',len(embeddings_cols))
print('len(loader_cols)',len(loader_cols))

print('-----------')

print('- wide_cols',wide_cols)
print('- interactive_features',interactive_features)
print('- D',D)
print('- embeddings_cols',embeddings_cols)
print('- loader_cols',loader_cols)

len wide_cols 23
len interactive_features 0
D 4194304
len embeddings_cols 21
len(loader_cols) 5
-----------
- wide_cols ['C1', 'banner_pos', 'site_id', 'site_domain', 'site_category', 'app_id', 'app_domain', 'app_category', 'device_id', 'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'hr', 'dayOfWeek']
- interactive_features []
- D 4194304
- embeddings_cols ['C1', 'banner_pos', 'site_id', 'site_domain', 'site_category', 'app_id', 'app_domain', 'app_category', 'device_model', 'device_type', 'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'hr', 'dayOfWeek']
- loader_cols ['id', 'date', 'hashed_wide', 'embedding_indexed', 'label']


=============

## make petastorm intemediate data

In [ ]:
from petastorm.spark import SparkDatasetConverter, make_spark_converter
from petastorm import TransformSpec

In [ ]:
!rm -r /content/petastorm # remove files so as to make everything inside petastorm are generated by current Python session
# set a cache directory for intermediate data, hold by "SparkDatasetConverter" object
spark.conf.set(SparkDatasetConverter.PARENT_CACHE_DIR_URL_CONF, "file:///content/petastorm/cache")
converter_train = make_spark_converter(train_df,dtype = 'float32') # int features will be kept as int
converter_test = make_spark_converter(test_df,dtype = 'float32')

rm: cannot remove '/content/petastorm': No such file or directory


Converting floating-point columns to float32
Converting floating-point columns to float32


In [ ]:
print(len(converter_train))
print(len(converter_test))
print(converter_train.cache_dir_url)
print(converter_test.cache_dir_url)

36210029
4218938
file:///content/petastorm/cache/20210223004709-appid-local-1614041176244-a6463189-9ba7-4726-a520-271040f9c203
file:///content/petastorm/cache/20210223005632-appid-local-1614041176244-b4c0dbfc-bb62-4f23-9b77-0680efe2326e


## save data

In [ ]:
# copy cached files to google drive
if full_data:
  !cp -r /content/petastorm    /content/drive/MyDrive/LAI_OFFER/BIG_DATA/AvazuCTR/WDL_FTRL/data_wDash/petastorm
else:
  !cp -r /content/petastorm   /content/drive/MyDrive/LAI_OFFER/BIG_DATA/AvazuCTR/WDL_FTRL/data_wDash/petastorm_filtered

In [ ]:
# copy converter object to google drive
file_name  = 'converter'
if full_data:
  converter_path = F"/content/drive/My Drive/LAI_OFFER/BIG_DATA/AvazuCTR/WDL_FTRL/data_wDash/petastorm/{file_name}.pickle"
else:
  converter_path = F"/content/drive/My Drive/LAI_OFFER/BIG_DATA/AvazuCTR/WDL_FTRL/data_wDash/petastorm_filtered/{file_name}.pickle"


# save 'converter_features_label' object (serialize)
import pickle
with open(converter_path, 'wb') as handle:
    pickle.dump(converter_train, handle, protocol=pickle.HIGHEST_PROTOCOL)
    pickle.dump(converter_test, handle, protocol=pickle.HIGHEST_PROTOCOL)

## make data loader and test it

In [ ]:
def get_transform_spec():
  return TransformSpec(func = None, selected_fields = loader_cols)

In [ ]:
batch_size = 32
for epoch in range(1):
  with converter_train.make_torch_dataloader(batch_size = batch_size, transform_spec = get_transform_spec(),num_epochs = 1, shuffle_row_groups = False, workers_count = 2) as train_loader, \
       converter_test.make_torch_dataloader(batch_size = batch_size, transform_spec = get_transform_spec(),num_epochs = 1,shuffle_row_groups = False, workers_count = 2) as test_loader:
       for batch_idx, row_batch in enumerate(train_loader):
         break
         

/usr/local/lib/python3.6/dist-packages/petastorm/arrow_reader_worker.py:53: FutureWarning: Calling .data on ChunkedArray is provided for compatibility after Column was removed, simply drop this attribute
  column_as_pandas = column.data.chunks[0].to_pandas()


In [ ]:
row_batch.keys()

dict_keys(['id', 'date', 'hashed_wide', 'embedding_indexed', 'label'])

In [ ]:
print(row_batch['hashed_wide'].shape)
print(row_batch['embedding_indexed'].shape)
print(row_batch['label'].shape)
print(row_batch['date'].shape)
print(row_batch['id'].shape)

torch.Size([32, 23])
torch.Size([32, 21])
torch.Size([32])
torch.Size([32])
torch.Size([32])


In [ ]:
print(row_batch['hashed_wide'].dtype) # float32, forced by vector Assembler
print(row_batch['embedding_indexed'].dtype)
print(row_batch['label'].dtype)
print(row_batch['date'].dtype)
print(row_batch['id'].dtype)

torch.float32
torch.float32
torch.float32
torch.int32
torch.float32


In [ ]:
row_batch['label']

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.])

In [ ]:
row_batch['hashed_wide'][:2,:]

tensor([[1726672.,  205682., 1214099., 1628535.,  371651., 1371062.,  276464.,
         1921245., 2513746.,  595326., 1839538.,  387265., 1681129.,  389681.,
         1631129.,  150290.,  232825.,  763427., 3510526., 2299873., 2955042.,
         2168740., 1544548.],
        [1726672.,  205682.,  782127., 3097264., 4192687., 1371062.,  276464.,
         1921245., 2513746., 2796683.,  330639.,  387265., 1681129., 1095835.,
         1631129.,  150290.,  232825.,  763427., 3510526., 2299873., 2955042.,
         2168740., 1544548.]])

In [ ]:
row_batch['embedding_indexed'][0,:]

tensor([  0.,   0.,  81.,  67.,   5.,   0.,   0.,   0., 653.,   0.,   0.,  10.,
          0.,   0.,   0.,   0.,   0.,   0.,   2.,  22.,   5.])

In [ ]:
row_batch['date']

tensor([1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024,
        1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024,
        1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024], dtype=torch.int32)